# Making the initial timetable

In [1]:
import pandas as pd
import numpy as np

In [2]:
proc_tt = pd.read_excel("Processed Data/encoded_timetable_data.xlsx").copy()
print(proc_tt.columns)
print("Shape of dataframe: ", proc_tt.shape)

Index(['Course', 'Code', 'Linked Course', 'Activity', 'Activity Type Name',
       'Planned Size', 'Real Size', 'Delivery Semester',
       'Teaching Week Pattern', 'Number Of Teaching Weeks', 'Scheduled Days',
       'Scheduled Start Time', 'Scheduled End Time', 'Duration', 'Total Hours',
       'Zone Name', 'room_name', 'Delivery Period', 'Normal Year Taken',
       'building', 'encoded_days', 'Abbreviated Course Name', 'time_periods',
       'duration_hours', 'total_duration_hours', 'course_classtype',
       'ws_groups', 'no_of_agg_ws_groups', 'aggregated_ws_groups',
       'sem1_pattern_one_hot', 'sem2_pattern_one_hot'],
      dtype='object')
Shape of dataframe:  (1259, 31)


## Cleaning data

In [3]:
# Drop MIGS course as we are not considering it for SoM
proc_tt.drop(proc_tt.index[proc_tt.Code == "MATH11220"].tolist(), inplace = True)

## Choosing weeks in semester 1 and semester 2 for timetabling

In [4]:
sem1_tt = proc_tt[proc_tt["sem1_pattern_one_hot"].str.contains('1')]
sem2_tt = proc_tt[proc_tt["sem2_pattern_one_hot"].str.contains('1')]
s2w3_tt = sem2_tt[sem2_tt["Teaching Week Pattern"].str.contains('s2w3')]
s2w4_tt = sem2_tt[sem2_tt["Teaching Week Pattern"].str.contains('s2w4')]

In [5]:
init_tt_s2w3 = np.zeros((9, 5), dtype = "O")
init_tt_s2w4 = np.zeros((9, 5), dtype = "O")

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
periods = ['1', '2', '3', '4', '5', '6', '7', '8', '9']

### Create initial timetable for any week

In [38]:
def create_init_tt(sem, week, days, periods, full_tt = proc_tt):
    sem_tt = full_tt[full_tt["sem"+str(sem)+"_pattern_one_hot"].str.contains('1')]
    sw_tt = sem_tt[sem_tt["Teaching Week Pattern"].str.contains('s'+str(sem)+'w'+str(week))]
    init_tt = pd.DataFrame(np.zeros((9, 5), dtype = "O"), index = periods, columns = days)
    
    # # For cross checking
    # sw_copy = sw_tt.copy()
    # sw_copy["time_periods"] = sw_copy["time_periods"].str.split(',')
    # sw_copy = sw_copy.explode("time_periods")

    # count = 0

    for d, day in enumerate(days):
        # sw_check = sw_copy[sw_copy["Scheduled Days"] == day]
        # day_tt = sw_check.drop_duplicates(subset = ["course_classtype", "room_name", "time_periods"])
        # count_check = 0
        # indices = []
        for t, period in enumerate(periods):
            # print(day, period)
            sw_day_time_tt = sw_tt[(sw_tt["Scheduled Days"] == day) & (sw_tt["time_periods"].str.contains(period))]
            sw_dt_tt_unique = sw_day_time_tt.drop_duplicates(subset = ["course_classtype", "room_name", "Scheduled Start Time"])
            # count_check += sw_dt_tt_unique.shape[0]
            # unique_indices = sw_dt_tt_unique.index.tolist()
            # indices.extend(unique_indices)
            classes_rooms = []
            for row in sw_dt_tt_unique.itertuples():
                classes_rooms.append([row.course_classtype, row.room_name])
                # count += 1
            
            init_tt.loc[period, day] = str(classes_rooms).replace(", ", "")

        # print(count_check, day_tt.shape[0])
        # print(np.sort(np.array(indices)), np.sort(np.array(day_tt.index.tolist())))

    # print(count, sw_copy.drop_duplicates(subset = ["course_classtype", "room_name", "Scheduled Days", "time_periods"]).shape[0])
    return init_tt
    
init_tt_s2w4 = create_init_tt(sem=2, week=4, days=days, periods=periods)
init_tt_s2w3 = create_init_tt(sem=2, week=3, days=days, periods=periods)

init_tt_s2w4.to_csv(f"Initial_timetables/init_tt_s2w4.csv", sep = " ")
init_tt_s2w3.to_csv(f"Initial_timetables/init_tt_s2w3.csv", sep = " ")

In [42]:
init_tt_s2w4.to_csv(f"Initial_timetables/init_tt_s2w4.csv", sep = ",")
init_tt_s2w3.to_csv(f"Initial_timetables/init_tt_s2w3.csv", sep = ",")